# Laboratorio 3
### Security Data Science
- Christian Pérez
- Andrei Portales

#### Preprocesamiento

In [35]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn import feature_extraction, tree, model_selection, metrics
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
%matplotlib inline
from yellowbrick.features import Rank2D
from yellowbrick.features import RadViz

In [36]:
df = pd.read_csv('VirusSample.csv')
df.head()

,file,api,class
0,7ff49f2f0912352416b05c010f35f402cc79feed,"IntersectRect,GetCurrentProcess,GetVersion",Virus
1,50cc6c99ec285d0db45dde07d8fdc18d9098c5b6,"GetCaretBlinkTime,CountClipboardFormats,GetCon...",Virus
2,f77c6bd4aebacd1a01d02e0cb20642ebf2d32929,"VarR8Pow,GetClipboardViewer,GetInputDesktop,Ge...",Virus
3,349c367c5b88fbb6cafae5d7109588d7250e16b5,"SetTraceCallback,CopyAcceleratorTableW,GetProc...",Virus
4,021f4aa86b520e1d606ab26699c35546bcd00c27,"SHLoadNonloadedIconOverlayIdentifiers,VarUI8Fr...",Virus


In [37]:
df.shape

(9795, 3)

In [38]:
# Preprosesamiento de datos

# Eliminamos las columnas que no nos interesan "file"
df = df.drop(['file'], axis=1)

# Eliminamos las filas que tienen valores nulos
df = df.dropna()

# change to lower case api and class
df['api'] = df['api'].str.lower()



In [39]:
# for every api in each row, we split it by comma and add it to a list
api_list = []
for row in df['api']:
    x = row.split(',')
    for y in x:
        api_list.append(y)

# delete duplicates in api_list
api_list = list(set(api_list))



In [40]:
print(len(api_list))
print(api_list[0])

7964
pathgetchartypew


In [41]:
# # create a column for each api and set the value to 1 if the api is present in the row
# for api in api_list:
#     for row in df['api']:
#         x = row.split(',')
#         if api in x:
#             df[api] = 1
#         else:
#             df[api] = 0



In [42]:
# df.head()

In [43]:
# for api in api_list:
#     df[api] = df["api"].apply(lambda x: 1 if api in x else 0)

In [44]:
df.head()

,api,class
0,"intersectrect,getcurrentprocess,getversion",Virus
1,"getcaretblinktime,countclipboardformats,getcon...",Virus
2,"varr8pow,getclipboardviewer,getinputdesktop,ge...",Virus
3,"settracecallback,copyacceleratortablew,getproc...",Virus
4,"shloadnonloadediconoverlayidentifiers,varui8fr...",Virus


In [45]:
# df = df.drop(['api'], axis=1)

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [47]:
tv = TfidfVectorizer(use_idf=True, smooth_idf=True, norm=None, tokenizer=lambda x: x.split(','))
tv_matrix = tv.fit_transform(df['api'])
tv_matrix = tv_matrix.toarray()


c:\Users\andre\Documents\Gitkraken\lab3-securitydata\venv\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [48]:
df2 = pd.DataFrame(np.round(tv_matrix,2), columns=tv.get_feature_names_out())

In [49]:
df2.head()

,??0?$basic_fstream@gu?$char_traits@g@std@@@std@@qae@pbdh@z,??0?$basic_ifstream@du?$char_traits@d@std@@@std@@qae@pb_whh@z,??0?$basic_ios@du?$char_traits@d@std@@@std@@iae@xz,??0?$basic_ios@du?$char_traits@d@std@@@std@@ieaa@xz,??0?$basic_istream@du?$char_traits@d@std@@@std@@qae@pav?$basic_streambuf@du?$char_traits@d@std@@@1@_n@z,??0?$basic_istream@du?$char_traits@d@std@@@std@@qeaa@peav?$basic_streambuf@du?$char_traits@d@std@@@1@_n@z,??0?$basic_ofstream@du?$char_traits@d@std@@@std@@qae@pb_whh@z,??0?$basic_ofstream@du?$char_traits@d@std@@@std@@qae@xz,??0?$basic_ostream@du?$char_traits@d@std@@@std@@qae@pav?$basic_streambuf@du?$char_traits@d@std@@@1@_n@z,??0?$basic_streambuf@du?$char_traits@d@std@@@std@@iae@xz,...,zwunloadkeyex,zwunlockfile,zwunmapviewofsection,zwvdmcontrol,zwwaitfordebugevent,zwwaitformultipleobjects,zwwaitforsingleobject,zwwritefile,zwwritefilegather,zwwriterequestdata
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
# modelTF_IDF.head()

In [51]:
# Train model with 70% train and 30% test using Random Forest
# X_train, X_test, y_train, y_test = model_selection.train_test_split(df, df['class'], test_size=0.3, random_state=0)
# clf = tree.DecisionTreeClassifier()
# clf.fit(X_train, y_train)

In [52]:
# Train model with 70% train and 30% test using Random Forest
X_train, X_test, y_train, y_test = model_selection.train_test_split(df2, df['class'], test_size=0.3, random_state=0)
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)


DecisionTreeClassifier()

In [53]:
y_pred = clf.predict(X_test)


In [54]:
# Validacion cruzada con k = 10
scores = model_selection.cross_val_score(clf, df2, df['class'], cv=10)
print(scores)

c:\Users\andre\Documents\Gitkraken\lab3-securitydata\venv\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


[0.93877551 0.94897959 0.93979592 0.92346939 0.92142857 0.93156282
 0.93462717 0.92951992 0.93667007 0.94484168]


In [55]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [56]:
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Confusion matrix: ', confusion_matrix(y_test, y_pred))
print('Classification report: ', classification_report(y_test, y_pred))

Accuracy:  0.9350119088125213
Confusion matrix:  [[  46    0    0    0    0    0    0    0   20    0    0]
 [   0    4    0    0    0    0    0    0   27    0    0]
 [   0    0  115    0    0    0    0    0   14    1    1]
 [   0    0    0    0    0    0    0    0    0    1    0]
 [   2    0    0    0    1    0    1    0    6    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    1    0    1]
 [   0    0    0    0    0    0    0    0    1    0    0]
 [   7    1    5    0    0    1    0    0 1812   11   17]
 [   0    0    0    0    0    0    0    0   29  679    3]
 [   0    0    0    0    0    0    0    0   41    0   91]]
Classification report:                precision    recall  f1-score   support

      Adware       0.84      0.70      0.76        66
       Agent       0.80      0.13      0.22        31
    Backdoor       0.96      0.88      0.92       131
       Crypt       0.00      0.00      0.00         1
  Downloader     

c:\Users\andre\Documents\Gitkraken\lab3-securitydata\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\andre\Documents\Gitkraken\lab3-securitydata\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\andre\Documents\Gitkraken\lab3-securitydata\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average

In [57]:
# Histogram-based Gradient Boosting 

from sklearn.ensemble import GradientBoostingClassifier

# Train model with 70% train and 30% test using Random Forest
X_train, X_test, y_train, y_test = model_selection.train_test_split(df2, df['class'], test_size=0.3, random_state=0)
clf = GradientBoostingClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

In [59]:
y_pred

array(['Trojan', 'Virus', 'Virus', ..., 'Trojan', 'Virus', 'Virus'],
      dtype=object)

In [ ]:
# Validacion cruzada con k = 10
scores = model_selection.cross_val_score(clf, df2, df['class'], cv=10)
print(scores)

In [61]:

print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Confusion matrix: ', confusion_matrix(y_test, y_pred))
print('Classification report: ', classification_report(y_test, y_pred))

Accuracy:  0.9343314052398775
Confusion matrix:  [[  47    0    0    0    0    2    0    1   16    0    0]
 [   1    4    0    0    0    0    0    0   26    0    0]
 [   0    0  115    0    0    0    0    0   14    1    1]
 [   0    0    0    0    0    0    0    0    0    1    0]
 [   2    0    0    0    0    0    0    0    7    0    1]
 [   0    0    0    0    0    0    0    0    1    1    0]
 [   0    0    0    0    0    0    0    0    0    1    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   6    0    2    1    2    0    1    0 1812   17   13]
 [   0    0    0    0    2    0    0    2   31  675    1]
 [   0    0    0    0    0    0    0    0   39    0   93]]
Classification report:                precision    recall  f1-score   support

      Adware       0.84      0.71      0.77        66
       Agent       1.00      0.13      0.23        31
    Backdoor       0.98      0.88      0.93       131
       Crypt       0.00      0.00      0.00         1
  Downloader     

c:\Users\andre\Documents\Gitkraken\lab3-securitydata\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\andre\Documents\Gitkraken\lab3-securitydata\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\andre\Documents\Gitkraken\lab3-securitydata\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [62]:
#  Support Vector Machine

from sklearn.svm import SVC

# Train model with 70% train and 30% test using Random Forest
X_train, X_test, y_train, y_test = model_selection.train_test_split(df2, df['class'], test_size=0.3, random_state=0)
clf = SVC()

clf.fit(X_train, y_train)

SVC()

In [64]:
y_pred = clf.predict(X_test)
y_pred

array(['Trojan', 'Virus', 'Virus', ..., 'Trojan', 'Virus', 'Virus'],
      dtype=object)

In [66]:
scores = model_selection.cross_val_score(clf, df2, df['class'], cv=10)
scores

c:\Users\andre\Documents\Gitkraken\lab3-securitydata\venv\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


array([0.94897959, 0.94489796, 0.93877551, 0.93979592, 0.92653061,
       0.93667007, 0.93258427, 0.92951992, 0.94586313, 0.94382022])

In [65]:

print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Confusion matrix: ', confusion_matrix(y_test, y_pred))
print('Classification report: ', classification_report(y_test, y_pred))

Accuracy:  0.9370534195304525
Confusion matrix:  [[  40    0    0    0    0    0    0   24    2    0]
 [   0    4    0    0    0    0    0   27    0    0]
 [   0    0  115    0    0    0    0   15    1    0]
 [   0    0    0    0    0    0    0    0    1    0]
 [   2    0    0    0    1    0    0    7    0    0]
 [   0    0    0    0    0    0    0    2    0    0]
 [   0    0    0    0    0    0    0    0    1    0]
 [   2    0    1    0    0    0    0 1839    7    5]
 [   0    0    0    0    1    0    0   31  679    0]
 [   0    0    0    0    0    0    0   56    0   76]]
Classification report:                precision    recall  f1-score   support

      Adware       0.91      0.61      0.73        66
       Agent       1.00      0.13      0.23        31
    Backdoor       0.99      0.88      0.93       131
       Crypt       0.00      0.00      0.00         1
  Downloader       0.50      0.10      0.17        10
  Ransomware       0.00      0.00      0.00         2
    Riskware     

c:\Users\andre\Documents\Gitkraken\lab3-securitydata\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\andre\Documents\Gitkraken\lab3-securitydata\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\andre\Documents\Gitkraken\lab3-securitydata\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

### Explicacion de resultados

Como podemos observar en los modelos implementados se logro un acurracy bueno. Podemos observar tambien en el analisis de datos que para ciertos tipos de malware, la precision no fue muy buena y son casi o son 0 esto debido a que para estos tipos hay muy pocos datos y por lo tanto el modelo no puede aprender de ellos. Por otro lado, para los tipos de malware que tienen mas datos, el modelo logra un acurracy bastante bueno.

Comparando con los resultados de los articulos podemos ver que estos son bastante similares a los que logramos con nuestros modelos, por lo que se logro entrenar bien a los modelos, solamente que en ciertos casos no se logro un buen accuracy debido a la poca cantidad de datos.